In [1]:
import meshio
import pygalmesh
import pygmsh
import numpy as np
import copy
import glob
from collections import Counter
import matplotlib.pyplot as plt
import os
import json
import shutil
import scipy.optimize as opt
from EnergyMinimization import *
import numba

# Initialisation

User settings: What are the continuum parameters we want? In 2D, we know that the elastic modulii are proportional to the microscopic spring constant. We also know that the continuum and microscopic momdulii are related by a lattice space: $\mu = O(1) k$, $k_c = k_d a$. Since I dont know any better, for know I will just set k to mu.

In [2]:
# Target mesh size:
target_a = 0.3
# continuum shear modulus:
mu=1
# Energetic penalty for volume change
#B=1000000
B=50000
# The Material Nonlinearity parameter, between 0 and 1
MatNon=0
#Energy penalty applied by the surfaces:
E=1000
khook = mu

Setting up the experiment:

In [3]:
# root folder for data
DataFolder=os.getcwd()+'/Data/Scratch/'
# Name of the current file
ScriptName="ElasticModuliiCalibration.ipynb"

In [4]:
try:
    os.mkdir(DataFolder)
except OSError:
    print ("Creation of the directory %s failed" % DataFolder)
else:
    print ("Successfully created the directory %s " % DataFolder)
    
# try and clear out the folder of vtk files and log files, if there was a previous run in it
for filename in glob.glob(DataFolder+'*.vtk')+glob.glob(DataFolder+'*.log'):
    file_path = os.path.join(DataFolder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
                
#Dump all the parameters to a file in the run folder        
f=open(DataFolder+"Parameters.log","w+")
datadict= { 
        "a":target_a, 
        "B":B,
        "mu":mu,
        "alpha": MatNon
}
json.dump(datadict,f)
f.close()

# and for good measure, dump a copy of this code into the data file too
shutil.copyfile(ScriptName,DataFolder+ScriptName)

Creation of the directory /home/jackbinysh/Code/ActiveElastocapillarity/Python/EnergyMinimization/Data/Scratch/ failed


'/home/jackbinysh/Code/ActiveElastocapillarity/Python/EnergyMinimization/Data/Scratch/ElasticModuliiCalibration.ipynb'

# Making the Mesh

Make the mesh, write it out to the folder. I am going to use the gmsh mesh

In [5]:
with pygmsh.occ.Geometry() as geom:
    geom.characteristic_length_max = target_a
    #geom.add_cylinder([0.0, 0.0, 0.0], 1)
    ellipsoid =geom.add_cylinder([0, 0.0, 0.0], [0.0, 0.0, 1], 1),
    #ellipsoid = geom.add_ellipsoid([0.0, 0.0, 0.0], [0.95, 0.95, 1.05])
    InputMesh = geom.generate_mesh()

/home/jackbinysh/miniconda3/lib/python3.8/site-packages/numpy/ctypeslib.py:521: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


Make the bond lists, make the oriented boundary triangles list, make the mapping from bonds to boundary triangles

In [6]:
interiorbonds,edgebonds,boundarytris, bidxTotidx, tetras= MakeMeshData3D(InputMesh)
bonds=np.concatenate((interiorbonds,edgebonds))
orientedboundarytris=OrientTriangles(InputMesh.points,boundarytris,np.array([0,0,0]))
boundarytris=orientedboundarytris

Indices of the points on the toplayer and bottom layer

In [7]:
TopLayer= np.where((InputMesh.points[:,2]>0.99))[0]
BottomLayer= np.where((InputMesh.points[:,2]<0.01))[0]

Write a copy of the input Mesh, for visualisation

In [8]:
cells=[ ("line", bonds ), ("triangle",boundarytris ), ("tetra",tetras)]
isbond=  np.ones(len(bonds))
isedgebond= np.concatenate( ( np.zeros(len(interiorbonds)),np.ones(len(edgebonds)) ) )
CellDataDict={'isedgebond':[isedgebond,np.zeros(len(boundarytris)),np.zeros(len(tetras))]
              ,'isbond':[isbond,np.zeros(len(boundarytris)),np.zeros(len(tetras))]}

OutputMesh=meshio.Mesh(InputMesh.points, cells, {},CellDataDict)
OutputMesh.write(DataFolder+"InitialMesh.vtk",binary=True) 

# Energy Minimization

In [11]:
def mycallback(xi):
    
    counter=len(history)
    tempP = xi.reshape((-1, 3))
    history.append(counter)
        
    print("iteration:"+"{0:0.1f}".format(counter))   
    OutputMesh.points = tempP  
    Name="z0_"+"{0:0.2f}".format(z0)+"_"+"{0:0d}".format(counter)+".vtk"
    CalibrationOutput3D(Name,DataFolder,OutputMesh,tempP,bonds,orientedboundarytris,tetras,r0_ij,khook,B,MatNon,TargetVolumes,z0,TopLayer,BottomLayer,E) 
    
    

In [ ]:
# make the preferred rest lengths of the interior springs
interiorpairs=InputMesh.points[interiorbonds]
interiorvecs = np.subtract(interiorpairs[:,0,:],interiorpairs[:,1,:])
InteriorBondRestLengths=np.linalg.norm(interiorvecs,axis=1)

# make the preferred rest lengths of the edge springs. Initially have the at g0=1, but then
#update them in the loop
edgepairs=InputMesh.points[edgebonds]
edgevecs = np.subtract(edgepairs[:,0,:],edgepairs[:,1,:])
EdgeBondRestLengths=np.linalg.norm(edgevecs,axis=1)
  
r0_ij=np.concatenate((InteriorBondRestLengths,EdgeBondRestLengths))

# The volume constraint is simply that the target volume should be the initial volume
TargetVolumes=Volume3D_tetras(InputMesh.points,tetras)

Pout_ij =InputMesh.points

#extension:
z0range=np.arange(1,2,0.1)

for z0 in z0range:
    
    print("Current z0"+"{0:0.2f}".format(z0))
    
    # minimize
    history=[]
    
    #ModuliiEnergy(P,TopLayer,BottomLayer,bondlist,tetras,r0_ij,z0,khook,B,E,MatNon,TargetVolumes): 
    Pout_ij = opt.minimize(ModuliiEnergy, Pout_ij.ravel()
                            ,callback=mycallback
                            ,options={'gtol':1e-02,'disp': True}  
                            ,args=(TopLayer
                                  ,BottomLayer
                                  ,bonds
                                  ,tetras
                                  ,r0_ij
                                  ,z0
                                  ,khook
                                  ,B
                                  ,E
                                  ,MatNon
                                  ,TargetVolumes)
                           ).x.reshape((-1, 3))
     
   
    Name="z0_"+"{0:0.2f}".format(z0)+".vtk"
    CalibrationOutput3D(Name,DataFolder,OutputMesh,Pout_ij,bonds,orientedboundarytris,tetras,r0_ij,khook,B,MatNon,TargetVolumes,z0,TopLayer,BottomLayer,E) 

Current z01.00
Optimization terminated successfully.
         Current function value: 69.989936
         Iterations: 0
         Function evaluations: 694
         Gradient evaluations: 1
Current z01.10
iteration:0.0
iteration:1.0
iteration:2.0
iteration:3.0
iteration:4.0
iteration:5.0
iteration:6.0
iteration:7.0
iteration:8.0
iteration:9.0
iteration:10.0
iteration:11.0
iteration:12.0
iteration:13.0
iteration:14.0
iteration:15.0
iteration:16.0
iteration:17.0
iteration:18.0
iteration:19.0
iteration:20.0
iteration:21.0
iteration:22.0
iteration:23.0
iteration:24.0
iteration:25.0
iteration:26.0
iteration:27.0
iteration:28.0
iteration:29.0
iteration:30.0
iteration:31.0
iteration:32.0
iteration:33.0
iteration:34.0
iteration:35.0
iteration:36.0
iteration:37.0
iteration:38.0
iteration:39.0
iteration:40.0
iteration:41.0
iteration:42.0
iteration:43.0
iteration:44.0
iteration:45.0
iteration:46.0
iteration:47.0
iteration:48.0
iteration:49.0
iteration:50.0
iteration:51.0
iteration:52.0
iteration:53.

iteration:152.0
iteration:153.0
iteration:154.0
iteration:155.0
iteration:156.0
iteration:157.0
iteration:158.0
iteration:159.0
iteration:160.0
iteration:161.0
iteration:162.0
iteration:163.0
iteration:164.0
iteration:165.0
iteration:166.0
iteration:167.0
iteration:168.0
iteration:169.0
iteration:170.0
iteration:171.0
iteration:172.0
iteration:173.0
iteration:174.0
iteration:175.0
iteration:176.0
iteration:177.0
iteration:178.0
iteration:179.0
iteration:180.0
iteration:181.0
iteration:182.0
iteration:183.0
iteration:184.0
iteration:185.0
iteration:186.0
iteration:187.0
iteration:188.0
iteration:189.0
iteration:190.0
iteration:191.0
iteration:192.0
iteration:193.0
iteration:194.0
iteration:195.0
iteration:196.0
iteration:197.0
iteration:198.0
iteration:199.0
iteration:200.0
iteration:201.0
iteration:202.0
iteration:203.0
iteration:204.0
iteration:205.0
iteration:206.0
iteration:207.0
iteration:208.0
iteration:209.0
iteration:210.0
iteration:211.0
iteration:212.0
iteration:213.0
iteratio

iteration:310.0
iteration:311.0
iteration:312.0
iteration:313.0
iteration:314.0
iteration:315.0
iteration:316.0
iteration:317.0
iteration:318.0
iteration:319.0
iteration:320.0
iteration:321.0
iteration:322.0
iteration:323.0
iteration:324.0
iteration:325.0
iteration:326.0
iteration:327.0
